# Setup

In [4]:
!pip install -q opendatasets

## Import libraries

In [5]:
import pandas as pd
import opendatasets as od
import numpy as np
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input

# Data Loading

In [6]:
od.download("https://www.kaggle.com/datasets/johnbergmann/captcha-image-dataset")

Dataset URL: https://www.kaggle.com/datasets/johnbergmann/captcha-image-dataset


100%|██████████| 39.5M/39.5M [00:02<00:00, 14.5MB/s]


In [7]:
DATA_URL = "/content/captcha-image-dataset/captchas"
train_dir = os.path.join(DATA_URL, "train")
test_dir = os.path.join(DATA_URL, "test")

# Data Preparation

In [8]:
BATCH_SIZE = 64
IMG_SIZE = (250,50)
N_LABELS = 36
D = 6 # number per image

In [9]:
files = []
labels = []

for filename in os.listdir(train_dir):
  files.append(os.path.join(train_dir, filename))
  labels.append(os.path.splitext(filename.split('_')[1])[0].lower())

In [10]:
df = pd.DataFrame({'file': files, 'label': labels})

In [11]:
df.head()

,file,label
0,/content/captcha-image-dataset/captchas/train/...,kmsvbm
1,/content/captcha-image-dataset/captchas/train/...,6zdebd
2,/content/captcha-image-dataset/captchas/train/...,teh6ts
3,/content/captcha-image-dataset/captchas/train/...,mxxcvk
4,/content/captcha-image-dataset/captchas/train/...,thsvpc


## Label Encoding

In [12]:
charset = "0123456789abcdefghijklmnopqrstuvwxyz"
chars_to_index = {char: i for i, char in enumerate(charset)}

In [13]:
def labels_to_indices(label):
  result = [chars_to_index[char] for char in label]
  return result

In [28]:
label_1 = []
label_2 = []
label_3 = []
label_4 = []
label_5 = []
label_6 = []

for label in df['label']:
  l1, l2, l3, l4, l5, l6 = labels_to_indices(label)
  label_1.append(l1)
  label_2.append(l2)
  label_3.append(l3)
  label_4.append(l4)
  label_5.append(l5)
  label_6.append(l6)

df['label_1'] = label_1
df['label_2'] = label_2
df['label_3'] = label_3
df['label_4'] = label_4
df['label_5'] = label_5
df['label_6'] = label_6

In [33]:
df.drop(columns='label', inplace=True)

In [34]:
df.head()

,file,label_1,label_2,label_3,label_4,label_5,label_6
0,/content/captcha-image-dataset/captchas/train/...,20,22,28,31,11,22
1,/content/captcha-image-dataset/captchas/train/...,6,35,13,14,11,13
2,/content/captcha-image-dataset/captchas/train/...,29,14,17,6,29,28
3,/content/captcha-image-dataset/captchas/train/...,22,33,33,12,31,20
4,/content/captcha-image-dataset/captchas/train/...,29,17,28,31,25,12
